COMMON SETUP

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(
      name="ml-playground-01",
      subscription_id="1a342961-1209-4b7b-9e44-318aeea934e8",
      resource_group="DataEngineering",
)

print(ws)

exp = Experiment(workspace=ws, name="udacity-project-01")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
from azureml.widgets import RunDetails

from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_target = ws.compute_targets["testing-123"]

Workspace.create(name='ml-playground-01', subscription_id='1a342961-1209-4b7b-9e44-318aeea934e8', resource_group='DataEngineering')
Workspace name: ml-playground-01
Azure region: eastus
Subscription id: 1a342961-1209-4b7b-9e44-318aeea934e8
Resource group: DataEngineering


START HYPER DRIVE BLOCK

In [34]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.01, 1),
        "--max_iter": choice([10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]),
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
from azureml.core import Environment

sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

from azureml.core import ScriptRunConfig

path_to_script = "." # "~/cloudfiles/code/Users/ballen/udacity/nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files"
src = ScriptRunConfig(source_directory=path_to_script,
                      script='train.py',
                      arguments=[],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="AUC",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=144,
    max_concurrent_runs=12,
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hd_config)


In [ ]:
# watch run
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

HYPERDRIVE CLEANUP

In [ ]:
hyperdrive_run.complete()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)

print(best_run.get_file_names())

# model = best_run.register_model(model_name='best_logistic', model_path='outputs/model.joblib')

START AUTOML BLOCK

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path)

In [28]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_hours=0.5,
    compute_target=compute_target,
    task="classification",
    model_explainability=True,
    training_data=ds,
    label_column_name="y",
    n_cross_validations=10,
    primary_metric="AUC_weighted",
    enable_onnx_compatible_models=False,
    enable_stack_ensemble=True,
    enable_voting_ensemble=True,
    iterations=144,
    enable_early_stopping=True,
    max_cores_per_iteration=-1,
    max_concurrent_iterations=12,
)

In [29]:
# Submit your automl run

automl_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-01,AutoML_b82a165d-32d7-40dc-8988-8fdd2e017f33,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [30]:
# watch run
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [31]:
automl_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-01,AutoML_b82a165d-32d7-40dc-8988-8fdd2e017f33,automl,Running,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the small

{'runId': 'AutoML_b82a165d-32d7-40dc-8988-8fdd2e017f33',
 'target': 'testing-123',
 'status': 'Completed',
 'startTimeUtc': '2021-06-21T20:24:04.94321Z',
 'endTimeUtc': '2021-06-21T20:43:46.878412Z',
 'properties': {'num_iterations': '144',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '10',
  'target': 'testing-123',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project-01","subscription_id":"1a342961-1209-4b7b-9e44-318aeea934e8","resource_group":"DataEngineering","workspace_name":"ml-playground-01","region":"eastus","compute_target":"testing-123","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":144,"primary_metric":"AUC_weighted","task_type":

AUTOML CLEANUP

In [32]:
automl_run.complete()

In [33]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
# print(fitted_model.steps)
print(type(automl_run))
print(type(best_run))
print(type(fitted_model))

model_name = best_run.properties['model_name']
print(model_name)
description = 'AutoML forecast example'
tags = None

model = automl_run.register_model(model_name = model_name, description = description, tags = tags)

Package:azureml-automl-runtime, training version:1.30.0, current version:1.28.0.post2
Package:azureml-core, training version:1.30.0, current version:1.28.0
Package:azureml-dataset-runtime, training version:1.30.0, current version:1.28.0
Package:azureml-defaults, training version:1.30.0, current version:1.28.0
Package:azureml-interpret, training version:1.30.0, current version:1.28.0
Package:azureml-mlflow, training version:1.30.0, current version:1.28.0
Package:azureml-pipeline-core, training version:1.30.0, current version:1.28.0
Package:azureml-telemetry, training version:1.30.0, current version:1.28.0
Package:azureml-train-automl-client, training version:1.30.0, current version:1.28.0
Package:azureml-train-automl-runtime, training version:1.30.0, current version:1.28.0


<class 'azureml.train.automl.run.AutoMLRun'>
<class 'azureml.core.run.Run'>
<class 'azureml.automl.runtime.shared.model_wrappers.PipelineWithYTransformations'>
AutoMLb82a165d352


EVERYTHING CLEANUP

In [ ]:
run.complete()